# Validating the Resnet Servable

Use this notebook to validate the servable that you just created, whether using Estimator or Keras!
This notebook can be used to validate that the servable you just trained works as intended by mocking part of the resnet client and feeding the input to the network.

**NOTE:** If you're building the ResNet servable from the Estimator API, the below validation only works if you trained ResNet with 'channels_last'. Not to fear! If the servable is validated using 'channels_last' mode, it will work with 'channels_first' mode.

In [0]:
from tensorflow.contrib import predictor
import cv2
import os

In [0]:
# Create a predict function from your saved model (servable)
# TODO: Use your own exported resnet serving directory and version number from the output of the last cell in
# `resnet_training_to_serving.ipynb` or `keras_resnet_serving.ipynb`.
SERVING_DIR = ???
VERSION = ???

predict_fn = predictor.from_saved_model(os.path.join(SERVING_DIR, str(VERSION)),
                                        signature_def_key='predict')

In [0]:
TOP_K = 5  # Set this depending on how many classes the server is expected to return

# Load a sample jpeg, and send it to the predict function (TF serving emulator)
IMAGE_PATH='client/cat_sample.jpg'

feature = cv2.imread(IMAGE_PATH)
feature = cv2.imencode('.jpg', feature)[1].tostring()
predictions = predict_fn(
    {'images': [feature]})

# Run some test cases
classes = predictions['classes']
probabilities = predictions['probabilities']
assert len(classes[0]) == TOP_K
assert len(probabilities[0]) == TOP_K
assert max(probabilities[0]) <= 1
assert max(probabilities[0]) >= 1.0 / TOP_K
assert min(probabilities[0]) >= 0
# Print your model output for inspection
print('Model passes unit test! Here are some predictions:\n')
print(predictions)

# If this succeeds, congratulations! Your model (probably) works!